In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import os
import importlib
import seaborn as sns

from collections import defaultdict
from scipy.stats.stats import pearsonr
from pandas.plotting import scatter_matrix
from scipy.stats import norm

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
for dirname, _, filenames in os.walk(r'C:\Users\loren\Desktop\DM_II'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Users\loren\Desktop\DM_II\datatest.txt
C:\Users\loren\Desktop\DM_II\datatest2.txt
C:\Users\loren\Desktop\DM_II\datatraining.txt


In [ ]:
df_test_1 = pd.read_csv(r'C:\Users\loren\Desktop\DM_II\datatest.txt',  header=0, parse_dates=True, squeeze=True)
df_test_2 = pd.read_csv(r'C:\Users\loren\Desktop\DM_II\datatest2.txt',  header=0, parse_dates=True, squeeze=True)
df_training = pd.read_csv(r'C:\Users\loren\Desktop\DM_II\datatraining.txt',  header=0, parse_dates=True, squeeze=True)

In [ ]:
df_test_1.sample(5)
df_test_2.sample(5)
df_training.sample(5)

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
2133,2015-02-03 23:31:59,20.890,25.080,0.0,556.80,0.003827,0
2081,2015-02-03 22:40:00,20.890,25.500,0.0,572.75,0.003892,0
944,2015-02-03 03:42:59,20.500,22.456,0.0,436.60,0.003343,0
1362,2015-02-03 10:40:59,21.870,28.252,493.0,1142.00,0.004584,1
852,2015-02-03 02:10:59,20.525,22.245,0.0,446.75,0.003317,0


,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
3727,2015-02-14 04:53:59,19.600,37.00,0.0,503.00,0.005225,0
59,2015-02-11 15:46:00,21.865,30.55,429.0,823.75,0.004958,1
3309,2015-02-13 21:55:59,20.290,31.29,0.0,510.00,0.004607,0
7016,2015-02-16 11:43:00,21.500,28.34,439.0,989.75,0.004495,1
2121,2015-02-13 02:08:00,21.000,24.39,0.0,578.00,0.003747,0


,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
4407,2015-02-07 19:17:00,20.100,24.066667,0.0,441.666667,0.003496,0
6454,2015-02-09 05:23:59,19.500,26.700000,0.0,460.333333,0.003738,0
6922,2015-02-09 13:12:00,21.500,33.163333,31.0,1243.333333,0.005266,0
528,2015-02-05 02:38:00,21.075,24.972500,0.0,440.750000,0.003855,0
2883,2015-02-06 17:53:00,21.675,21.175000,433.0,798.750000,0.003388,1


In [ ]:
from pandas import concat
df = concat([df_test_1, df_training, df_test_2], ignore_index=True)
df.head()

df.to_csv('dataframe.csv')

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,2015-02-02 14:19:00,23.7000,26.272,585.200000,749.200000,0.004764,1
1,2015-02-02 14:19:59,23.7180,26.290,578.400000,760.400000,0.004773,1
2,2015-02-02 14:21:00,23.7300,26.230,572.666667,769.666667,0.004765,1
3,2015-02-02 14:22:00,23.7225,26.125,493.750000,774.750000,0.004744,1
4,2015-02-02 14:23:00,23.7540,26.200,488.600000,779.000000,0.004767,1


In [ ]:
df.columns

Index(['date', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio',
       'Occupancy'],
      dtype='object')

In [ ]:
import datetime

df['Hours'] = pd.to_datetime(df['date']).dt.hour

In [ ]:
df.sample(5)

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Hours
12024,2015-02-12 11:04:00,23.945,22.575,291.75,828.75,0.004150,0,11
17257,2015-02-16 02:17:00,20.390,27.340,0.00,851.50,0.004047,0,2
15829,2015-02-15 02:29:00,20.000,34.145,0.00,538.00,0.004941,0,2
9003,2015-02-09 03:29:00,19.390,26.790,0.00,467.00,0.003725,0,3
16604,2015-02-15 15:23:59,21.700,31.025,50.00,658.50,0.004985,0,15


In [ ]:
df = df.drop('date', axis = 1)
df.sample(5)

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Hours
15977,20.10,32.290000,0.0,568.00,0.004700,0,4
13390,22.50,24.840000,717.0,577.00,0.004185,1,9
17095,20.29,29.890000,0.0,733.00,0.004400,0,23
166,22.60,25.000000,433.0,836.25,0.004238,1,17
15195,19.79,36.163333,24.0,509.00,0.005167,0,15


In [ ]:
df['Time_blocks'] = df['Hours'] // 6

In [ ]:
df.head(20)

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Hours,Time_blocks
0,23.7000,26.2720,585.200000,749.200000,0.004764,1,14,2
1,23.7180,26.2900,578.400000,760.400000,0.004773,1,14,2
2,23.7300,26.2300,572.666667,769.666667,0.004765,1,14,2
3,23.7225,26.1250,493.750000,774.750000,0.004744,1,14,2
4,23.7540,26.2000,488.600000,779.000000,0.004767,1,14,2
5,23.7600,26.2600,568.666667,790.000000,0.004779,1,14,2
6,23.7300,26.2900,536.333333,798.000000,0.004776,1,14,2
7,23.7540,26.2900,509.000000,797.000000,0.004783,1,14,2
8,23.7540,26.3500,476.000000,803.200000,0.004794,1,14,2
9,23.7360,26.3900,510.000000,809.000000,0.004796,1,14,2


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
le = LabelEncoder()
num_classes = le.fit_transform(df['Time_blocks'])
print(num_classes[:1000])

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 

In [ ]:
df.dtypes

Temperature      float64
Humidity         float64
Light            float64
CO2              float64
HumidityRatio    float64
Occupancy          int64
Hours              int64
Time_blocks        int64
dtype: object

In [ ]:
class_name = 'Occupancy'
columns2remove = ['Hours', 'Humidity']
df.drop(columns2remove, inplace=True, axis =1)

In [ ]:
df.head(20)

,Temperature,Light,CO2,HumidityRatio,Occupancy,Time_blocks
0,23.7000,585.200000,749.200000,0.004764,1,2
1,23.7180,578.400000,760.400000,0.004773,1,2
2,23.7300,572.666667,769.666667,0.004765,1,2
3,23.7225,493.750000,774.750000,0.004744,1,2
4,23.7540,488.600000,779.000000,0.004767,1,2
5,23.7600,568.666667,790.000000,0.004779,1,2
6,23.7300,536.333333,798.000000,0.004776,1,2
7,23.7540,509.000000,797.000000,0.004783,1,2
8,23.7540,476.000000,803.200000,0.004794,1,2
9,23.7360,510.000000,809.000000,0.004796,1,2


In [ ]:
def prepare_dataset(df, class_name):
    df = remove_missing_values(df)
    numeric_columns = get_numeric_columns(df)
    rdf = df.copy(deep=True)
    df, feature_names, class_values = one_hot_encoding(df, class_name)
    real_feature_names = get_real_feature_names(rdf, numeric_columns, class_name)
    rdf = rdf[real_feature_names + (class_values if isinstance(class_name, list) else [class_name])]
    features_map = get_features_map(feature_names, real_feature_names)

    return df, feature_names, class_values, numeric_columns, rdf, real_feature_names, features_map

def remove_missing_values(df):
    for column_name, nbr_missing in df.isna().sum().to_dict().items():
        if nbr_missing > 0:
            if column_name in df._get_numeric_data().columns:
                mean = df[column_name].mean()
                df[column_name].fillna(mean, inplace=True)
            else:
                mode = df[column_name].mode().values[0]
                df[column_name].fillna(mode, inplace=True)
    return df

def get_numeric_columns(df):
    numeric_columns = list(df._get_numeric_data().columns)
    return numeric_columns

def get_real_feature_names(rdf, numeric_columns, class_name):
    real_feature_names = [c for c in rdf.columns if c in numeric_columns and c != class_name]
    real_feature_names += [c for c in rdf.columns if c not in numeric_columns and c != class_name]
    return real_feature_names

def one_hot_encoding(df, class_name):
    dfX = pd.get_dummies(df[[c for c in df.columns if c != class_name]], prefix_sep='=')
    class_name_map = {v: k for k, v in enumerate(sorted(df[class_name].unique()))}
    dfY = df[class_name].map(class_name_map)
    df = pd.concat([dfX, dfY], axis=1)
    feature_names = list(dfX.columns)
    class_values = sorted(class_name_map)
    return df, feature_names, class_values

def get_features_map(feature_names, real_feature_names):
    features_map = defaultdict(dict)
    i = 0
    j = 0

    while i < len(feature_names) and j < len(real_feature_names):
        if feature_names[i] == real_feature_names[j]:
            features_map[j][feature_names[i]] = j
            i += 1
            j += 1
        elif feature_names[i].startswith(real_feature_names[j]):
            features_map[j][feature_names[i]] = j
            i += 1
        else:
            j += 1
    return features_map

In [ ]:
res = prepare_dataset(df, class_name)
df, feature_names, class_values, numeric_columns, rdf, real_feature_names, features_map = res
df.head()

,Temperature,Light,CO2,HumidityRatio,Time_blocks,Occupancy
0,23.7000,585.200000,749.200000,0.004764,2,1
1,23.7180,578.400000,760.400000,0.004773,2,1
2,23.7300,572.666667,769.666667,0.004765,2,1
3,23.7225,493.750000,774.750000,0.004744,2,1
4,23.7540,488.600000,779.000000,0.004767,2,1


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(df.values)

In [ ]:
attributes = [col for col in df.columns if col != class_name]
X = df[attributes].values
y = df[class_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)

In [ ]:
X_train.shape, X_test.shape

((14392, 5), (6168, 5))

In [ ]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB

In [ ]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.9641699092088197
F1-score [0.97616737 0.92784851]
              precision    recall  f1-score   support

           0       1.00      0.95      0.98      4743
           1       0.87      1.00      0.93      1425

    accuracy                           0.96      6168
   macro avg       0.93      0.98      0.95      6168
weighted avg       0.97      0.96      0.97      6168



In [ ]:
df[numeric_columns].head()

,Temperature,Light,CO2,HumidityRatio,Occupancy,Time_blocks
0,23.7000,585.200000,749.200000,0.004764,1,2
1,23.7180,578.400000,760.400000,0.004773,1,2
2,23.7300,572.666667,769.666667,0.004765,1,2
3,23.7225,493.750000,774.750000,0.004744,1,2
4,23.7540,488.600000,779.000000,0.004767,1,2


In [ ]:
X_train_num, X_test_num, y_train, y_test = train_test_split(
    df[[c for c in numeric_columns if c != class_name]], df[class_name], 
       test_size=0.3, random_state=100, stratify=df[class_name])

In [ ]:
clf = GaussianNB()
clf.fit(X_train_num, y_train)
y_pred = clf.predict(X_test_num)
print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

GaussianNB(priors=None, var_smoothing=1e-09)

Accuracy 0.9641699092088197
F1-score [0.97616737 0.92784851]
              precision    recall  f1-score   support

           0       1.00      0.95      0.98      4743
           1       0.87      1.00      0.93      1425

    accuracy                           0.96      6168
   macro avg       0.93      0.98      0.95      6168
weighted avg       0.97      0.96      0.97      6168



In [ ]:
pip install scikit-plot

In [ ]:
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall
from scikitplot.metrics import plot_cumulative_gain
from scikitplot.metrics import plot_lift_curve

In [ ]:
y_score_1 = clf_1.predict_proba(X_test)
plot_roc(y_test, y_score_1)
plt.show()

In [ ]:
plot_precision_recall(y_test, y_score_1)
plt.show()

In [ ]:
plot_cumulative_gain(y_test, y_score_1)
plt.show()

In [ ]:
plot_lift_curve(y_test, y_score_1)
plt.show()